In [7]:
import sklearn as sk
from sklearn.neural_network import BernoulliRBM
import numpy as np
import matplotlib.pyplot as plt
from sklearn.externals import joblib


The MNIST data set is stored in train_set_data.npy (training data) and train_set_label.npy (labels).
It can be loaded using numpy.load. I have split off 10000 training examples as cross-validation set, so
$N_{train}$ is only 50000

In [8]:
train_data=np.load('train_set_data.npy')
train_label=np.load('train_set_label.npy')
print('shape of training data: (Ntrain,Nfeature)={0}'.format(train_data.shape))
#rescale the data
td=train_data*256.0/255.0
#sweeps 30 times through training data using bathcsize of 200
#some hyper parameters (see webpage below for details)
nhidden=100#td.shape[1] number of hidden layers; the smaller the faster is training
batchsize=200
niter=50
rbm=BernoulliRBM(n_components=nhidden,verbose=True,batch_size=batchsize,n_iter=niter)

#let's initialize a second RBM and train it only very shortly; 
#we can then compare the two RBM
#sweeps 1 times through training data using batchsizes of 10 images
rbm_stupid=BernoulliRBM(n_components=nhidden,verbose=True,batch_size=10,n_iter=1)
#%matplotlib inline


shape of training data: (Ntrain,Nfeature)=(50000, 784)


### Train and save the RBM

In [4]:
rbm.fit(td)
joblib.dump(rbm, 'rbm_mnist_NH{0}.pkl'.format(nhidden))

KeyboardInterrupt: 

### BernoulliRBM
is a class provided in sklearn.neural_network.BernoulliRBM. Below is a snapshot of the web-site


In [3]:
import IPython
url = 'http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.BernoulliRBM.html'
iframe = '<iframe src=' + url + ' width=950 height=1000></iframe>'
IPython.display.HTML(iframe)


## load two RBMs and compare them

In [10]:

nhidden=td.shape[1]
f1='rbm_uninit_mnist_NH{0}.pkl'.format(nhidden)
f2='rbm_mnist_NH{0}.pkl'.format(nhidden)
rbm_uninit = joblib.load(f1) 
rbm = joblib.load(f2) 

## Perform Gibbs sampling using the trained RBM
We start from a randomly chosen training sample

In [ ]:
import random

#chose a trainingsample at random
ind=random.randint(0,td.shape[0])
v=td[ind,:]
vt=v
vut=v

#you can also start from a random pixel image by uncommenting the line below:
#v=np.random.random_sample(v.shape)

N=1000 #number of daydream steps
plt.figure(1)
plt.figure(2)

plt.ion()

plt.imshow(np.reshape(1.0-v,(28,28)),cmap='gray')
for n in range(N): 
    vt=rbm.gibbs(vt)
    vut=rbm_uninit.gibbs(vut)
    #plt.subplot(N1,N2,n+1)
    plt.figure(1)
    plt.clf()
    plt.title('well trained RBM')
    plt.imshow(np.reshape(1.0-vt,(28,28)),cmap='gray')
    plt.tight_layout()
    
    plt.figure(2)
    plt.pause(0.1)
    plt.clf()
    plt.title('stupid RBM')
    plt.imshow(np.reshape(1.0-vut,(28,28)),cmap='gray')
    plt.tight_layout()
    plt.show()
    plt.draw()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python2.7/lib-tk/Tkinter.py", line 1489, in __call__
    return self.func(*args)
  File "/usr/lib/python2.7/lib-tk/Tkinter.py", line 536, in callit
    func(*args)
  File "/usr/lib/pymodules/python2.7/matplotlib/backends/backend_tkagg.py", line 363, in idle_draw
    self.draw()
  File "/usr/lib/pymodules/python2.7/matplotlib/backends/backend_tkagg.py", line 348, in draw
    FigureCanvasAgg.draw(self)
  File "/usr/lib/pymodules/python2.7/matplotlib/backends/backend_agg.py", line 451, in draw
    self.figure.draw(self.renderer)
  File "/usr/lib/pymodules/python2.7/matplotlib/artist.py", line 55, in draw_wrapper
    draw(artist, renderer, *args, **kwargs)
  File "/usr/lib/pymodules/python2.7/matplotlib/figure.py", line 1034, in draw
    func(*args)
  File "/usr/lib/pymodules/python2.7/matplotlib/artist.py", line 55, in draw_wrapper
    draw(artist, renderer, *args, **kwargs)
  File "/usr/lib/pymodules/python

For comparison, here is the Gibbs sampling for an almost untrained RBM

## Some questions:
1. When should we stop training? 
2. What is the meaning of the pseudo-likelihood value? Is it the log of a probability?
3. How many steps of Gibbs-sampling are done before a data-point is collected for the evaluation of the negative statistics (see http://www.cs.toronto.edu/~hinton/absps/guideTR.pdf)
4. How many __should__ be done?